### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/finals/baselines
Obtaining file:///project/finals/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
Requirement already up-to-date: tensorflow-gpu==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)


In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11710866553381032129,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17682822122196959827
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17200648020038197876
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
%cd ..
logdir = "Training/Logs/DDQN/" + datetime.now().strftime("%Y%m%d-%H%M%S")

/project/finals


In [4]:
cicddos2019 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
cicdos2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
cicids2017 = pd.read_feather("/project/datasets/clean-ids-collection/cic-ids2017/clean/cicids2017.feather")
data = pd.concat([cicddos2019], ignore_index=True)
print(len(data.columns))
data.head(5)


69


,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.583496,28337.111328,98168.0,3.0,9529897.0,351582.62500,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.417969,121314.914062,420255.0,4.0,9493930.0,351541.09375,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.000000,0.000000,62416.0,62416.0,36373056.0,0.00000,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.000000,0.000000,62413.0,62413.0,36372292.0,0.00000,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.000000,0.000000,62409.0,62409.0,36372216.0,0.00000,36372216.0,36372216.0,Benign


In [5]:
cicdos2017_features = ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"]
cicids2017_features = ["Protocol", "Avg Bwd Segment Size", "Packet Length Max", "Bwd Packet Length Min", "Fwd IAT Mean"]
cicddos2019_features = ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"]
important_features = cicddos2019_features + ["Label", "Timestamp"] # Adding Timestamp for custom environment logic
important_features = list(set(important_features))
print(len(important_features))

7


In [6]:
removable_features = data.columns ^ important_features
print(removable_features.shape)
skinny_data = data.drop(labels=removable_features, axis='columns')

(62,)


In [7]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138007 entries, 0 to 138006
Data columns (total 69 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Protocol                  138007 non-null  int8          
 1   Timestamp                 138007 non-null  datetime64[ns]
 2   Flow Duration             138007 non-null  int32         
 3   Total Fwd Packets         138007 non-null  int16         
 4   Total Backward Packets    138007 non-null  int16         
 5   Fwd Packets Length Total  138007 non-null  float32       
 6   Bwd Packets Length Total  138007 non-null  float32       
 7   Fwd Packet Length Max     138007 non-null  float32       
 8   Fwd Packet Length Min     138007 non-null  float32       
 9   Fwd Packet Length Mean    138007 non-null  float32       
 10  Fwd Packet Length Std     138007 non-null  float32       
 11  Bwd Packet Length Max     138007 non-null  float32       
 12  Bw

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,138007.000000,1.380070e+05,138007.000000,138007.000000,1.380070e+05,1.380070e+05,138007.000000,138007.000000,138007.000000,138007.000000,...,138007.000000,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,1.380070e+05,138007.0
mean,11.592050,1.035663e+07,13.812169,6.636576,4.018388e+03,5.102365e+03,249.255005,132.856659,161.374619,37.344303,...,10.525227,-1.175840e+07,7.345767e+04,3.066105e+04,1.140839e+05,5.479827e+04,3.445545e+06,3.282423e+05,3.770921e+06,3153571.0
std,5.607782,2.853154e+07,70.298079,99.503208,5.148026e+04,1.880163e+05,419.245300,234.480453,241.070892,116.256744,...,62.973954,1.112129e+08,5.181675e+05,2.394353e+05,6.899427e+05,4.797951e+05,1.158544e+07,2.158710e+06,1.230299e+07,11206258.0
min,0.000000,1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.408238e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,6.000000,1.178000e+03,2.000000,0.000000,1.800000e+01,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,17.000000,2.489200e+04,3.000000,2.000000,8.600000e+01,0.000000e+00,43.000000,30.000000,39.000000,0.000000,...,1.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,17.000000,2.999380e+06,8.000000,2.000000,2.064000e+03,1.660000e+02,440.000000,152.000000,349.500000,14.433757,...,5.000000,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
max,17.000000,1.199987e+08,20444.000000,31700.000000,1.526642e+07,5.842950e+07,32120.000000,2131.000000,3015.290527,2221.556152,...,18766.000000,1.480000e+03,4.050800e+07,1.100562e+07,4.050800e+07,4.050800e+07,1.192194e+08,6.961402e+07,1.192194e+08,119219448.0


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [8]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [9]:
data.drop('Timestamp', inplace=True, axis=1)
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)

Benign           95095
DrDoS_NTP        13687
TFTP             13027
Syn               6713
UDP               2158
MSSQL             1763
DrDoS_UDP         1174
UDP-lag           1097
DrDoS_MSSQL        927
DrDoS_DNS          552
LDAP               484
DrDoS_SSDP         413
DrDoS_SNMP         392
Portmap            215
DrDoS_LDAP         174
NetBIOS             73
DrDoS_NetBIOS       51
UDPLag              10
WebDDoS              2
Name: Label, dtype: int64


In [10]:
x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

Label
Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
      

In [11]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(77240, 67)
(77240,)
Validation
(0, 67)
(0,)
Testing
(8582, 67)
(8582,)


In [12]:
print(x_train.shape)

(77240, 67)


In [13]:
print(x_train.index)
print(y_train.index)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

Int64Index([120674,  45987, 123133,  32037, 100187,  27212, 104682,  38559,
             38847,  63573,
            ...
             78892,  76032,  27952,  69918,  11949,  29929,  93718,  75675,
             73743,  17842],
           dtype='int64', length=77240)
Int64Index([120674,  45987, 123133,  32037, 100187,  27212, 104682,  38559,
             38847,  63573,
            ...
             78892,  76032,  27952,  69918,  11949,  29929,  93718,  75675,
             73743,  17842],
           dtype='int64', length=77240)


In [14]:
# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


In [15]:
print(x_train.index)
print(y_train.index)

RangeIndex(start=0, stop=77240, step=1)
RangeIndex(start=0, stop=77240, step=1)


In [16]:
print(y_train)

       Label
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
...      ...
77235    1.0
77236    1.0
77237    1.0
77238    1.0
77239    1.0

[77240 rows x 1 columns]


In [17]:
print(y_train.iloc[17162,:])

Label    0.0
Name: 17162, dtype: float32


In [18]:
print(y_train.index)
print(x_train.index)

RangeIndex(start=0, stop=77240, step=1)
RangeIndex(start=0, stop=77240, step=1)


### Custom environment

In [19]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(67,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [20]:
def ids_ddqn():
    env = IdsEnv(images_per_episode=4096)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        verbose=1, 
        learning_rate=0.00025, 
        buffer_size=1000000, 
        learning_starts=50000, 
        batch_size=32, 
        gamma=0.99, 
        train_freq=4, 
        target_network_update_freq=30000, # 10k to 30k
        exploration_fraction=0.01, # 0.1 to 0.01
        exploration_initial_eps=1.0, 
        exploration_final_eps=0.05, 
        tensorboard_log=logdir,
        full_tensorboard_log=True,
    ).learn(
        total_timesteps=int(1.0e6),
        log_interval=10,
        
    )

    model.save('ddqn_cicddos2019_allfeat.pkl')
    env.close()
    
    return model

start_time = time.time()
ddqn_model = ids_ddqn()
print("DDQN Training Time:", time.time() - start_time)








Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 10       |
| mean 100 episode reward | 1.61e+03 |
| steps                   | 36864    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 20       |
| mean 100 episode reward | 1.64e+03 |
| steps                   | 77824    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 30       |
| mean 100 episode reward | 1.64e+03 |
| steps                   | 118784   |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 40       |
| mean 100 episode reward | 1.66e+03 |
| steps                   | 159744   |
-----

In [21]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=4096, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:%
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        FPR = (float(FP) / (TN + FP)) # 1 - specificity
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))

ids_eval(ddqn_model)

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 60.40573627142357%
Precision: 21.193751457216134%
Recall/TPR/Sensitivity: 98.27027027027026%
FPR: 44.17145844223732%
F1 score: 0.3486766398158803


##### 